# Riot API로 받지 못하는 정보 OP.GG에서 크롤링하기

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

### 각 라인의 탑 티어 챔피언 구하기

In [2]:
url = 'https://www.op.gg/champion/statistics'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [3]:
item_df = pd.read_csv('../00. data/items.csv', index_col = 0)
spell_df = pd.read_csv('../00. data/spell.csv', index_col = 0)
rune_df = pd.read_csv('../00. data/runes.csv', index_col = 0)

In [4]:
# 아이템키로 아이템 이름 불러오는 함수
def searchItem(itemKey):
    item = item_df['name'][item_df['name'][item_df['key'] == itemKey].index[0]]
    return item 

In [5]:
# 영어이름의 스펠을 한글로 바꿔주는 함수
def translation_spell(eng):
    spell = spell_df['name'][spell_df['name'][spell_df['eng'] == eng].index[0]]
    return spell

In [6]:
# 룬키로 룬 이름 불러오는 함수
def searchRune(runeKey):
    rune = rune_df['name'][rune_df['name'][rune_df['key'] == runeKey].index[0]]
    return rune

In [7]:
# 태그 지우는 함수
def removeTags(strings):
    strings = re.sub('<.+?>', '', str(strings), 0).strip().replace("'",'')
    return strings

In [8]:
# OP.GG 아이템이미지의 키를 구하는 함수 + 룬
def findItem(itemLink):
    items = re.findall(r'\d\d\d\d.png', str(itemLink))
    items = [item.strip('.png') for item in items]
    
    return items

In [9]:
# OP.GG 스킬이미지의 영문 이름을 구하는 함수
def findSkill(link):
    name = re.findall(r'spell/.*png', str(link))
    name = ''.join(name).strip('spell/').strip('.png')
    return name

In [10]:
lanes = ['TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT']

In [11]:
for lane in lanes:
    top5 = soup.find(attrs={'class':f'tabItem champion-trend-tier-{lane}'}).select('.champion-index-table__name')[:10] # 10개만 가져오기
    print(f'{lane}의 최고티어 챔피언:',removeTags(top5))

TOP의 최고티어 챔피언: [Irelia, Camille, Fiora, Sett, Sylas, Jax, Aatrox, Wukong, Viego, Tahm Kench]
JUNGLE의 최고티어 챔피언: [Lee Sin, Xin Zhao, Elise, Viego, Karthus, Nocturne, Kindred, RekSai, Shaco, Fiddlesticks]
MID의 최고티어 챔피언: [Katarina, LeBlanc, Talon, Zed, Yasuo, Sylas, Anivia, Aurelion Sol, Pantheon, Viego]
ADC의 최고티어 챔피언: [Ezreal, Ziggs, Vayne, Samira, KogMaw, Ashe, Jinx, Swain, KaiSa, Jhin]
SUPPORT의 최고티어 챔피언: [Leona, Lulu, Thresh, Senna, Xerath, Blitzcrank, Nautilus, Maokai, Karma, Pyke]


### 원하는 챔피언의 기록 가져오기

In [12]:
# 샘플은 탑라인의 이렐리아
lane = 'top'
champion ='irelia'
url = f'https://www.op.gg/champion/{champion}/statistics/{lane}'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [13]:
win_rank = soup.select('.champion-stats-trend-rank')[0].find('b').text + soup.select('.champion-stats-trend-rank')[0].find('span').text
pick_rank = soup.select('.champion-stats-trend-rank')[1].find('b').text + soup.select('.champion-stats-trend-rank')[1].find('span').text
print('승률 순위:', win_rank, '픽률 순위:', pick_rank)

승률 순위: 28/55 픽률 순위: 1/55


### 원하는 챔피언의 추천 세팅 

- 스펠 , 스킬트리, 아이템 트리

In [14]:
for i in range(2):
    tmp = soup.select('ul.champion-stats__list')[i].find_all('img')[0] 
    tmp2 = soup.select('ul.champion-stats__list')[i].find_all('img')[1]
    pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick')[i].find('strong').text
    pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick')[i].find('span').text
    winRate = soup.select('.champion-overview__stats.champion-overview__stats--win')[i].find('strong').text
    print('추천 스펠 : '+ translation_spell(findSkill(tmp)), translation_spell(findSkill(tmp2)), ' | 픽률 : '+ pickRate, '총표본 수 : ' + pickTotal, '승률 : '+ winRate)
skillSeq = []
for i in range(3):
    skill = findSkill(soup.select('ul.champion-stats__list')[2].find_all('img')[i*2])
    skillSeq.append(skill.lower().split(champion)[1])
pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick')[2].find('strong').text
pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick')[2].find('span').text
winRate = soup.select('.champion-overview__stats.champion-overview__stats--win')[2].find('strong').text
print('스킬 트리: ',' -> '.join(skillSeq).upper(),' | 픽률:', pickRate, '총표본 수 : ' + pickTotal, '승률:', winRate)    
for i in range(10):
    tmp = soup.select('td.champion-overview__data.champion-overview__border.champion-overview__border--first')[i].find_all('img')
    keys = findItem(tmp)
    items = [searchItem(int(key)) for key in keys]
    items = ', '.join(items)
    pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick.champion-overview__border')[i].find('strong').text
    pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick.champion-overview__border')[i].find('span').text
    winRate = soup.select('.champion-overview__stats.champion-overview__stats--win.champion-overview__border')[i].find('strong').text
    if i < 2:
        msg = '시작아이템 :'
    elif i >= 2 and i < 7:
        msg = '추천 빌드 :'
    else:
        msg = '신발 :'
    print(msg + items, '| 픽률 : '+ pickRate,'총표본 수 : ' + pickTotal, '승률 : '+ winRate)

추천 스펠 : 점멸 순간이동  | 픽률 : 80.62% 총표본 수 : 32,784 승률 : 50.59%
추천 스펠 : 점멸 점화  | 픽률 : 18.19% 총표본 수 : 7,396 승률 : 50%
스킬 트리:  Q -> E -> W  | 픽률: 49.07% 총표본 수 : 9,388 승률: 62.39%
시작아이템 :도란의 검, 체력 물약 | 픽률 : 79.86% 총표본 수 : 31,783 승률 : 50.98%
시작아이템 :부패 물약 | 픽률 : 15.65% 총표본 수 : 6,228 승률 : 47.93%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 구인수의 격노검 | 픽률 : 21.83% 총표본 수 : 3,847 승률 : 63.89%
추천 빌드 :몰락한 왕의 검, 선혈포식자, 스테락의 도전 | 픽률 : 9.95% 총표본 수 : 1,753 승률 : 64.29%
추천 빌드 :몰락한 왕의 검, 신성한 파괴자, 스테락의 도전 | 픽률 : 9.48% 총표본 수 : 1,670 승률 : 61.86%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 마법사의 최후 | 픽률 : 9.1% 총표본 수 : 1,603 승률 : 62.45%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 죽음의 무도 | 픽률 : 6.51% 총표본 수 : 1,148 승률 : 59.32%
신발 :판금 장화 | 픽률 : 68.94% 총표본 수 : 25,099 승률 : 52.24%
신발 :헤르메스의 발걸음 | 픽률 : 27.94% 총표본 수 : 10,171 승률 : 54.85%
신발 :명석함의 아이오니아 장화 | 픽률 : 2.37% 총표본 수 : 862 승률 : 60.32%


- 룬

In [15]:
mainRune = {'Precision' : '정밀', 'Domination' : '지배', 'Sorcery' : '마법', 'Resolve' : '결의', 'Inspiration' : '영감'}

In [16]:
# 첫번째 추천룬
total_info = soup.select_one('.tabItem.ChampionKeystoneRune-1')
rate_info = total_info.select('.champion-overview__stats.champion-overview__stats--pick')
rec_runes = soup.select('.champion-stats-summary-rune__name')
rune_info = soup.select('.champion-stats-summary-rune__rate')
eng_rune = rec_runes[0].text.split(' + ')
kor_rune = mainRune[eng_rune[0]] + ' + ' + mainRune[eng_rune[1]]
pickRate = rune_info[0].find('strong').text
winRate = rune_info[0].find_all('span')[-1].text
print(kor_rune, '| 픽률 : ' + pickRate, '승률 : ' + winRate)
for i in range(2): 
    pickRate = rate_info[i].find('strong').text
    winRate= rate_info[i].find_all('strong')[-1].text
    pickTotal = rate_info[i].find_all('span')[1].text
    print('추천룬 | 픽률 : ' + pickRate, '승률 : ' + winRate, '총 표본수 : ' + pickTotal)
    for k in range(33):
        if i == 1:
            k += 33
        link = total_info.find_all('img')[k].attrs['src']
        if 'e_grayscale' in str(link):
            pass
        else:
            print(searchRune(int(''.join(findItem(link)))))

정밀 + 결의 | 픽률 : 71.08% 승률 : 50.71%
추천룬 | 픽률 : 22.82% 승률 : 49.09% 총 표본수 : 9,278
정밀
정복자
승전보
전설: 민첩함
최후의 저항
결의
뼈 방패
소생
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9
추천룬 | 픽률 : 8.01% 승률 : 51.9% 총 표본수 : 3,256
정밀
정복자
승전보
전설: 민첩함
최후의 저항
결의
뼈 방패
불굴의 의지
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9


In [17]:
# 두번째 추천 룬
rate_info = soup.select_one('.ChampionKeystoneRune-2').select('.champion-overview__stats.champion-overview__stats--pick')
eng_rune = rec_runes[1].text.split(' + ')
kor_rune = mainRune[eng_rune[0]] + ' + ' + mainRune[eng_rune[1]]
pickRate = rune_info[1].find('strong').text
winRate = rune_info[1].find_all('span')[-1].text
print(kor_rune, '| 픽률 : ' + pickRate, '승률 : ' + winRate)
for i in range(2):
    pickRate = rate_info[i].find('strong').text
    winRate= rate_info[i].find_all('strong')[-1].text
    pickTotal = rate_info[i].find_all('span')[1].text
    print('추천룬 | 픽률 : ' + pickRate, '승률 : ' + winRate, '총 표본수 : ' + pickTotal)
    for k in range(33):
        if i == 1:
                k += 33
        link = soup.select_one('.ChampionKeystoneRune-2').find_all('img')[k].attrs['src']
        if 'e_grayscale' in str(link):
            pass
        else:
            print(searchRune(int(''.join(findItem(link)))))

정밀 + 영감 | 픽률 : 16.59% 승률 : 49.12%
추천룬 | 픽률 : 3.9% 승률 : 46.53% 총 표본수 : 1,584
정밀
정복자
승전보
전설: 민첩함
최후의 저항
영감
비스킷 배달
시간 왜곡 물약
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9
추천룬 | 픽률 : 1.62% 승률 : 48.03% 총 표본수 : 660
정밀
정복자
승전보
전설: 민첩함
최후의 저항
영감
비스킷 배달
시간 왜곡 물약
공격속도 + 10%
적응형 능력치 + 9
마법저항력 + 8


### 매치업

In [18]:
url = f'https://www.op.gg/champion/{champion}/statistics/{lane}/matchup'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [19]:
for i in range(30):
    champ = soup.select('.champion-matchup-list__champion')[i].find('span').text
    winRate = soup.select('.champion-matchup-list__champion')[i].find_all('span')[1].string
    winRate = [item.strip() for item in winRate if str(item)]
    winRate = ''.join(winRate)
    play_rate = soup.select('.champion-matchup-list__totalplayed')[i].find('span').text
    play_num = soup.select('.champion-matchup-list__totalplayed')[i].find('small').text
    print(f'{champion.capitalize()} vs {champ} 상대승률 :',winRate)
    print(f'통계량: {play_num} 통계 비율 {play_rate}')

Irelia vs Sett 상대승률 : 49.5%
통계량: 20,845 통계 비율 8.02%
Irelia vs Fiora 상대승률 : 48.42%
통계량: 16,942 통계 비율 6.52%
Irelia vs Sylas 상대승률 : 51.19%
통계량: 14,620 통계 비율 5.62%
Irelia vs Camille 상대승률 : 50.35%
통계량: 10,317 통계 비율 3.97%
Irelia vs Renekton 상대승률 : 52.76%
통계량: 9,881 통계 비율 3.80%
Irelia vs Jax 상대승률 : 48.48%
통계량: 9,647 통계 비율 3.71%
Irelia vs Malphite 상대승률 : 51.22%
통계량: 9,327 통계 비율 3.59%
Irelia vs Aatrox 상대승률 : 54.29%
통계량: 8,944 통계 비율 3.44%
Irelia vs Gwen 상대승률 : 54.15%
통계량: 8,388 통계 비율 3.23%
Irelia vs Garen 상대승률 : 48.09%
통계량: 8,330 통계 비율 3.20%
Irelia vs Jayce 상대승률 : 56.47%
통계량: 7,117 통계 비율 2.74%
Irelia vs Darius 상대승률 : 53%
통계량: 6,849 통계 비율 2.63%
Irelia vs Riven 상대승률 : 49.33%
통계량: 6,749 통계 비율 2.60%
Irelia vs Akali 상대승률 : 56.53%
통계량: 5,921 통계 비율 2.28%
Irelia vs Wukong 상대승률 : 47.85%
통계량: 5,317 통계 비율 2.05%
Irelia vs Shen 상대승률 : 49.3%
통계량: 5,136 통계 비율 1.98%
Irelia vs Tahm Kench 상대승률 : 47.26%
통계량: 5,021 통계 비율 1.93%
Irelia vs Ornn 상대승률 : 50.16%
통계량: 4,892 통계 비율 1.88%
Irelia vs Volibear 상대승률 : 50.82%
통계량: